# Week 5: Single Cell RNA Sequencing

## Learning objectives

In this module, you will learn to:

1. Understand how the limitations of scRNA-seq technology contribute to:
    - The importance of cleaning our data
    - The statistical distribution of our data
2. Write code to:
    - Filter `AnnData` (and `DataFrames`) based on column values
    - Calculate and plot row sums of count data
    - Perform transformations and analyze single-cell data
    - Create histograms of specific genes

## Setup

Before we start, ensure you have all the necessary libraries installed. Run the following commands to install `scanpy` and `AnnData`.


In [ ]:
!pip install -q AnnData scanpy

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
from anndata import AnnData

## How do we prepare single-cell data for analysis?

Preparing single-cell RNA sequencing (scRNA-seq) data is an important step to ensure accurate results. Raw data often contains noise due to technical issues (e.g., issues while preparing samples), so it’s essential to filter out low-quality cells and genes. Key steps include calculating quality control (QC) metrics, removing unwanted cells or genes based on these metrics, and normalizing the data to adjust for differences in sequencing depth. This process helps make the data more reliable for further analysis, like clustering or identifying cell types, leading to clearer biological insights.


### Analyzing Single-Cell Data

A typical scRNA-seq dataset includes:

- Thousands of cells (observations).
- Gene expression levels across thousands of genes (features).

This results in a high-dimensional matrix, where each cell's gene expression profile is a point in this vast space.

### The `PBMC3K` Dataset

In this module, we will be working with the `pbmc3k` dataset, which is an scRNA-seq dataset made publicly available by 10x Genomics. It consists of roughly 3,000 Peripheral Blood Mononuclear Cells (PBMCs) from a healthy donor.

PBMCs are a diverse mixture of blood cells, including lymphocytes (T cells, B cells, NK cells), monocytes, and dendritic cells, each playing a crucial role in the immune system. This dataset is often used as a benchmark in scRNA-seq data analysis because it encompasses a variety of cell types, making it ideal for exploring dimensionality reduction techniques and clustering algorithms.

(The data is freely available from this [webpage](https://support.10xgenomics.com/single-cell-gene-expression/datasets/1.1.0/pbmc3k).)

We'll use a Python library called [`scanpy`](https://scanpy.readthedocs.io/en/stable/) to analyze and visualize single-cell gene expression data, especially from single-cell RNA sequencing (scRNA-seq) experiments. This library stores this data in a new Python object called `AnnData`, which is like a `pandas` `DataFrame` but with more information.
Before looking at the data itself, let's get familiar with `AnnData`.

### What is `AnnData`?

Up until now, you have been introduced to dictionaries, `numpy` arrays, and `pandas` `DataFrames`.
All of these data structures can now be combined into a single, more complex data structure that typically stores scRNA-seq data called an `Anndata` (annotated data) object.

`AnnData` is like a super-organized binder or folder for biological data, especially used in single-cell research (like looking at gene activity in thousands of cells at once).

![AnnData Image](https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg)
Image can be found at [here](https://raw.githubusercontent.com/scverse/anndata/main/docs/_static/img/anndata_schema.svg).

**Imagine a binder with 4 main sections:**

1. **`X`** – the main table
    - This is like a big spreadsheet where:
      - Rows = cells (like each cell from a person or mouse)
      - Columns = genes (like each gene you measured)
      - The values = how much each gene is "on" in each cell

2. **`obs`** – info about each cell
    - Think of sticky notes on each row. They might say:
      - “This cell came from a tumor”
      - “This cell is a T-cell”
      - “This cell came from patient A”

3. **`var`** – info about each gene
    - Sticky notes on each column. Things like:
      - “This gene is related to cancer”
      - “This gene is mitochondrial”

4. **`uns`** – the junk drawer
    - A place to store anything else you need:
      - Color maps for plotting
      - Settings for visualizations
      - Results from analyses
      
We will focus on `X`, `obs`, and `var` in this module.

**Bonus sections (optional but useful):
- **`obsm`** – extra information for each cell (like 2D coordinates for plots)
- **`varm`** – extra information for each gene
- **`layers`** – multiple versions of the data (e.g., raw vs. normalized)

**Summary:** `AnnData` is a data container that keeps everything neatly organized when working with tons of biological data. It’s perfect for single-cell experiments where you have info on thousands of cells and genes. Easy to share, analyze, and visualize!

Additional details on `Anndata` objects can be found [here](https://anndata.readthedocs.io/en/latest/).


### Loading the `PBMC3K` Dataset

Now that we've seen a bit more about the component parts of `AnnData` objects, we're ready to load the `pbmc3k` dataset from `scanpy`.
We will start with the raw dataset (unprocessed), loaded by the following code (code retrieved from [here](https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html)):

In [ ]:
adata = sc.read_10x_mtx(
    os.path.join(os.getcwd(),"data"),  # The directory with the `.mtx` file
    var_names="gene_symbols",  # Use gene symbols for the variable names (variables-axis index)
    cache=True,  # Save the data for faster subsequent reading
)

Now that we have retrieved our scRNA-seq data into an `AnnData` object, let's access specific components of the data and make some interpretations.

Recall that for an `AnnData` object, the `X` attribute stores the main table, which for scRNA-seq data is a count matrix. Here is a simplified example of what the count matrix might look like:

```
          Gene_1  Gene_2  Gene_3  Gene_4  Gene_5 ...
Cell_1      5       0       2       0       0
Cell_2      0       3       1       0       0
Cell_3      7       1       0       4       3
Cell_4      0       0       0       0       0
...
```

The rows are the cells and the columns are the genes. Each value represents the number of unique transcript molecules (UMI counts) detected for a specific gene in a single cell. Unique transcript molecules (UMI counts) = the deduplicated count of original RNA molecules per gene per cell, providing a more accurate measure of gene expression.

Let's explore the actual `adata.X` table we've loaded in.

In [ ]:
# Here is the shape of the data; how many cells and genes are represented in the dataset?
print(adata.X.shape)

# To print out just the values from the count matrix, we can use:
print(adata.X.toarray())

---
**Q*1: Calculate the percentage of non-zeroes in the count matrix. Why is the matrix so sparse (most of the elements are zero)?**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE
percent_zeros = ...
print(f"Percent of non zeros: {percent_zeros:.2f}%")

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer:

---

Next, let's look at the `adata.obs` and `adata.var` components, which store information about the cells and genes in the dataset, respectively.

In [ ]:
adata.obs

`adata.obs` contains one column with *cell barcodes*, unique identifiers (typically RNA or DNA sequences) that are attached to individual cells to track them over time.

In [ ]:
adata.var

`adata.var` shows useful metadata about the genes: their name and id.

Before making changes to our dataset, we might want to keep the original raw data. Preserving raw data in single-cell RNA sequencing ensures reproducibility, accuracy, and flexibility for downstream analyses by maintaining unaltered counts for reference and transformation.

In [ ]:
# Copies the entire AnnData object, preserving/storing raw counts in `adata.raw.X`
adata.raw = adata.copy()

## Data cleaning

Next, let's do some data cleaning on the raw data we have!

We want to exclude cells that have poor-quality data, such as cells with:

- **Library Size**: The library size is defined as the total sum of counts across all genes for a given cell. A cell with a low library size may indicate poor-quality data, possibly due to issues like cell damage, inefficient cell capture, or incomplete RNA extraction.

- **Expressed Genes**: The number of unique genes with non-zero expression in the cell. Cells with few expressed genes are likely to be of poor quality, as it suggests that the full spectrum of the cell's RNA population was not detected (due to cell damage or improper experimental procedures).

- **High Mitochondrial Gene Expression**: If a cell shows high levels of mitochondrial gene expression relative to other genes, it may indicate that the cell is damaged or dying. For modest damage, the holes in the cell membrane are too small to allow for mitochondria to escape. As a result, high mitochondrial gene expression is often a marker of poor-quality cells, which may need to be excluded from further analysis due to compromised integrity.

For more information about quality control, refer to this [webpage](https://bioconductor.org/books/3.12/OSCA/quality-control.html).

The following data processing and visualization exercises in this section are based on this [webpage](https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering-2017.html).

### Exploration: highly-expressed genes

Let's look at the top 20 expressed genes for our dataset:

In [ ]:
# Sum expression counts across all cells for each gene (sum over rows for each column)
gene_counts = np.array(adata.X.sum(axis=0)).flatten()  # axis = 0 sums down the rows, so per gene total counts

# Create a DataFrame summarizing gene names and their total counts
gene_summary = pd.DataFrame({
    'gene': adata.var_names,     # Gene symbols/names from the AnnData object
    'gene_counts': gene_counts   # Total counts summed across all cells for each gene
})

# Calculate the total sum of all gene counts across all genes (total transcript molecules)
total_counts_all = gene_counts.sum()

# Calculate the fraction (%) that each gene contributes to the total counts
gene_summary['fraction'] = gene_summary['gene_counts'] / total_counts_all * 100

# Sort the DataFrame by the fraction in descending order and show the top 20 genes
gene_summary.sort_values(by='fraction', ascending=False).head(20)

We just completed a manual computation of the total sum of all gene counts across all genes (total transcript molecules) and the fraction (%) that each gene contributes to the total counts.

Let's compare our results to the `sc.pl.highest_expr_genes()` function, a built-in tool in `scanpy` that quickly identifies and visualizes the genes with the highest expression levels across the dataset. It calculates the total counts for each gene and displays the top genes contributing most to the overall transcriptome. Refer to the documentation [here](https://scanpy.readthedocs.io/en/1.11.x/api/generated/scanpy.pl.highest_expr_genes.html).

Now, let's see how to use it.

In [ ]:
# Show those genes that yield the highest fraction of counts in each single cell, across all cells.
sc.pl.highest_expr_genes(adata, n_top=20)

### Filtering Low Library Size and Few Expressed Genes

Let's try filtering by library size. We will choose a minimum of 1000 total gene count expressed in a cell.

In [ ]:
# 1. Count how many genes are expressed in each cell (non-zero values)
n_genes_per_cell = adata.X.sum(axis=1)

# 2. Create a boolean mask - which rows are cells that contain a minimum of 1000 genes
cells_with_1000_genes = n_genes_per_cell >= 1000

# 3. Filter the AnnData object
adata_filtered = adata[cells_with_1000_genes, :]

# Let's look at the results of this filtering
print(f"Original number of cells: {adata.shape[0]}")
print(f"Filtered number of cells: {adata_filtered.shape[0]}")

---
**Q*2: Filter the cells by unique expressed genes with a minimum of 2000, storing the filtered data in a variable called `adata_filtered_2000`.**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE

# 1. Count how many genes are expressed in each cell (non-zero values)


# 2. Create a boolean mask - which rows are cells that contain a minimum of 2000 genes


# 3. Filter the AnnData object


# Let's look at the results of this filtering
print(f"Original number of cells: {adata.shape[0]}")
print(f"Filtered number of cells: {adata_filtered_2000.shape[0]}")

---
#### **QC Metrics**

In single-cell RNA sequencing (scRNA-seq), mitochondrial gene expression serves as an important quality control (QC) metric because high mitochondrial content often indicates stressed, damaged, or dying cells. Healthy cells typically have low mitochondrial gene expression, while cells under stress or apoptosis may show elevated mitochondrial gene expression.

Let's analyze the QC results of our dataset.

In [ ]:
# Annotate the group of mitochondrial genes as "MT" - assigns a boolean value for each gene
adata_filtered.var["mt"] = adata_filtered.var_names.str.startswith("MT-")

sc.pp.calculate_qc_metrics(
    adata_filtered, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)

After running `calculate_qc_metrics()`, the QC metrics will be stored in `adata.obs`, which contains metrics like:
- **`n_genes_by_counts`**: The number of genes detected in each cell.
- **`total_counts`**: The total number of reads (or counts) in each cell.
- **`total_counts_mt`**: The total number of reads (or counts) coming from mitochondrial genes for each cell.
- **`pct_counts_mt`**: The percentage of counts coming from mitochondrial genes for each cell.

The `adata.obs` field should now be populated!

In [ ]:
adata_filtered.obs

Let's try viewing information of a specific sample cell:

In [ ]:
name_of_the_sample = 'TTTGCATGAGAGGC-1'

adata_filtered.obs[adata_filtered.obs.index == name_of_the_sample]

---
**Q*3: Calculate the average value for each column and print: `n_genes_by_counts`,	`total_counts`,	`total_counts_mt`,	`pct_counts_mt`.**

> Hint: Refer to the documentation for `DataFrame.mean()` [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html#pandas-dataframe-mean).

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE

ave_n_genes_by_counts = 
ave_total_counts = 
ave_total_counts_mt = 
ave_pct_counts_mt = 

print(f"average n_genes_by_counts: {ave_n_genes_by_counts}")
print(f"average total_counts: {ave_total_counts}")
print(f"average total_counts_mt: {ave_total_counts_mt}")
print(f"average pct_counts_mt: {ave_pct_counts_mt}")

---
Let's try visualizing our results by creating a histogram of the total gene counts of the cells:

In [ ]:
plt.hist(adata_filtered.obs['n_genes_by_counts'], bins=50, color='skyblue', edgecolor='black')
plt.title(f'Histogram of n_genes_by_counts')
plt.xlabel('n_genes_by_counts')
plt.ylabel('Number of Cells')

plt.tight_layout()
plt.show()

---
**Q*4: Create a histogram for `total_counts`, `total_counts_mt`, and `pct_counts_mt`. Set `bins=50`.**

> Hint: Try to use a `for` loop to reduce redundancy in your code.

In [ ]:
# YOUR CODE HERE

# SOLUTION

metrics = ['total_counts',	'total_counts_mt', 'pct_counts_mt']

for metric in metrics:
  plt.hist(adata_filtered.obs[metric], bins=50, color='skyblue', edgecolor='black')
  plt.title(f'Histogram of {metric}')
  plt.xlabel(metric)
  plt.ylabel('Number of Cells')

  plt.tight_layout()
  plt.show()

---

We can filter for large gene counts by slicing the `AnnData` object:

In [ ]:
adata_final = adata_filtered[adata_filtered.obs.n_genes_by_counts < 2500, :]

---
**Q*5: Try filtering by percentage of mitochondrial genes expressed less than 5%. How many entries were there before and after filtering?**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE


---
### Normalization

Normalization allows for a fairer comparison of gene expression across cells with different total counts, removes biases introduced by differences in sequencing depth, and prepares the data for meaningful downstream analysis. The `sc.pp.normalize_total()` function in particular scales the counts to a target total count, which helps standardize the data.

In [ ]:
adata_final.obs

In [ ]:
sc.pp.normalize_total(adata_final, target_sum=1e4)

Note that we chose `target_sum=1e4`. Here are the following factors to consider when choosing this value:
- **Target Sum of 1e4**: A common choice for many single-cell RNA-seq datasets.
- **Choose Based on Sequencing Depth**: Higher sequencing depth might require a larger target sum.
- **Visualize Your Data**: Check the distribution of total counts and pick a target sum that reflects the typical sequencing depth in your dataset.

---
**Q*6: Redo the QC metric calculations and print the resulting `obs`.**

> Hint: After printing out `obs`, check that `total_counts` for each cell roughly equals the value that you normalized to (1e4).

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE

...

adata_final.obs

---
You can also make modifications to the `AnnData` object and store it back in the same object as a layer. This allows you to store different versions or transformations of the data while retaining the original.

In [ ]:
# Access the cell expression matrix values of the anndata object, log normalize it,
# and store a normalized version of the data as a log_transformed layer

adata_final.layers["log_transformed"] = np.log1p(adata_final.X).toarray()
adata_final.layers["log_transformed"]

### What does single-cell data look like?

We will use the **preprocessed data** for the `PBMC3K` dataset for the rest of the module. Data retrieved from [here](https://scanpy.readthedocs.io/en/stable/generated/scanpy.datasets.pbmc3k_processed.html) and [here](https://scanpy.readthedocs.io/en/stable/tutorials/basics/clustering-2017.html).

In [ ]:
pbmc3k_data = sc.datasets.pbmc3k()

Let's plot the histogram of a specific gene. For better visualization, we want to plot for a gene that is expressed in `n = 2000` numbers of cells:

In [ ]:
gene_ids = pbmc3k_data.var['gene_ids']

n = 2000

# Count non-zero values for each gene (column) using getnnz(axis=0)
non_zero_counts = pbmc3k_data.X.getnnz(axis=0)

# Filter genes based on non-zero count exceeding the threshold 'n'
genes_above_threshold = gene_ids.index[non_zero_counts > n]

# Print the filtered genes
genes_above_threshold

In [ ]:
gene = genes_above_threshold[0]

expression = pbmc3k_data[:, gene].X.toarray().flatten()
plt.hist(expression, bins=10, color='skyblue', edgecolor='black')
plt.title(f'Histogram of {gene} Expression')
plt.xlabel('Expression level')
plt.ylabel('Number of cells')
plt.show()

Variability in gene expression across cells for specific genes arises from a complex interplay of transcriptional regulation, chromatin state, post-transcriptional control, stochasticity, and external signals, all of which can be unique to each cell's context, state, or environment.

---
**Q*7: Plot the histogram of five other genes. What do you notice about their distribution and why is it like that?**

> Hint: Use a `for` loop to prevent redundant code.

<span style="background-color: #FFD700">**Write your code below**</span>

In [ ]:
# YOUR CODE HERE


<span style="background-color: #FFD700">**Write your answer below**</span>


Answer:

---

**Q*8: Calculate the total gene count per cell. Then, calculate the mean total gene count.**
> Hint: Code to sum along an axis: `matrix.sum(axis=...)`. Think about what are the different values for `axis`, and what do they mean?

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE

# Calculate total counts per cell
total_counts_per_cell = 

# Calculate mean
mean_count = 

print(total_counts_per_cell)
print(mean_count)

---
**Q*9: Plot a histogram of the total counts per cell. Compare bin size `5` vs. `50`. Which is the better option to visualize the distribution of the number of cells vs total gene count?**

**Add a vertical line for the mean using `matplotlib.axvline()`. Add labels and a legend.**

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE

# Plot histogram of total counts per cell


# Plot vertical lines for mean, median, and IQR


# Customize the plot


# Show the plot



<span style="background-color: #FFD700">**Write your answer below**</span>

Answer here:

---

## **Graded Exercises: (8 marks)**

**GQ*1. (2 marks) What is `AnnData` in Python, and what are its key components?**

<span style="background-color: #FFD700">**Write your answer below**</span>

Answer:

---

**GQ*2. (2 marks) Explain how you would scale the raw RNA-seq counts in `AnnData` to prepare data for downstream analysis, such as for PCA or clustering, which we’ll explore in future modules.**

> Hint: There is a built-in function for `scanpy`. Look up the documentation for details and how it works.

<span style="background-color: #FFD700">**Write your answer below**</span>


Answer:

---

**GQ*3. (2 marks) Using a new `AnnData` object shown below, normalize the data (`target_sum=1e4`), perform QC calculations, and print the results.**


The following code retrieves data from [Gene Expression - Feature / cell matrix (per-sample)](https://www.10xgenomics.com/datasets/10k_Human_PBMC_5p_v3_Ultima).

In [ ]:
adata_gq = sc.read_10x_mtx(
    os.path.join(os.getcwd(), "data_graded"),  # The directory with the `.mtx` file
    var_names="gene_symbols",  # Use gene symbols for the variable names (variables-axis index)
    cache=True,  # Write a cache file for faster subsequent reading
)
sc.pp.filter_cells(adata_gq, min_genes=1000)

<span style="background-color: #FFD700">**Write your code below**</span>


In [ ]:
# YOUR CODE HERE


**GQ*4. (2 marks) Using the new `AnnData` object shown above, display the top 20 expressed genes.**

<span style="background-color: #FFD700">**Write your code below**</span>

In [ ]:
# YOUR CODE HERE


## Conclusion

Single-cell data often comes in the form of count matrices stored in `AnnData`, with a focus on sparsity, where many gene expressions are zero for each cell. Preprocessing single-cell data involves filtering low-quality cells and genes, calculating quality control metrics, and normalizing the data to ensure comparability across cells. These steps are essential to prepare the data for downstream analysis, enabling researchers to uncover meaningful biological patterns and better understand cellular processes in health and disease.

In this module, you've gained hands-on experience in preprocessing, manipulating, and analyzing single-cell RNA-seq data using the `AnnData` object. These skills are essential for preparing data, performing quality control, and conducting in-depth analyses of transcriptomic data.